In [4]:
import pyro
import pyro.distributions as dist
import torch
import numpy as np
import matplotlib.pyplot as plt
import pyro.infer.mcmc as mcmc
import torch.distributions


def bayesian_pca(data, latent_dim, mask): #Pass in mask directly into the model 
    # Define model parameters
    n, p = data.shape
    sigma = pyro.sample("sigma", dist.Uniform(0., 10.))
    mu = torch.zeros(p)
    covariance = sigma * torch.eye(p)
    

#     Z_nl = pyro.sample("Z", dist.MultivariateNormal(torch.zeros(latent_dim), torch.eye(latent_dim)).expand((n,)).to_event(1))
    
#     W_lp = pyro.sample("W", dist.MultivariateNormal(torch.zeros(p), torch.eye(p)).expand((latent_dim,)).to_event(1))
    Z_mean = pyro.param("Z_mean", torch.zeros(n, latent_dim))
    Z_cov = pyro.param("Z_cov", torch.eye(latent_dim))


    W_mean = pyro.param("W_mean", torch.zeros(latent_dim, p))
    W_cov = pyro.param("W_cov", torch.eye(p))
    
    
    Z_mean.data = torch.zeros(n, latent_dim)
    Z_cov.data = torch.eye(latent_dim)
    W_mean.data = torch.zeros(latent_dim, p)
    W_cov.data = torch.eye(p)

    Z_nl = pyro.sample("Z", dist.MultivariateNormal(Z_mean, Z_cov))
    W_lp = pyro.sample("W", dist.MultivariateNormal(W_mean, W_cov))
    
    mean = Z_nl @ W_lp

    # sample observed variables from a multivariate Gaussian with mean "mean" and covariance "covariance"
    
    #Implement mask on our data to avoid those probabilities. 

    X = pyro.sample("X", dist.Normal(mean, sigma).mask(mask), obs=data)
    
    
    return X

In [6]:
def bayesian_pca_mcmc(data, latent_dim, num_samples, warmup_steps):

    pyro.clear_param_store()   
    # Run MCMC

    kernel = mcmc.NUTS(bayesian_pca)
    mcmc_run = mcmc.MCMC(kernel, num_samples=num_samples, warmup_steps=warmup_steps)

    N, P = data.shape
    mask_rows = torch.randperm(N)[:20]
    mask_cols = torch.randperm(P)[:5]

    mask = torch.ones(N,P)

    for i in mask_rows:
        for j in mask_cols:
            mask[i,j] = 0
    random_mask = mask.bool() #Boolean Tensor for mask

    end_block_mask = torch.ones(N,P)
    end_block_mask[-30:, -3:] = 0
    end_block_mask = end_block_mask.bool()


    test_mask = torch.ones(N,P)
    test_mask = test_mask.bool()

    #Apply MCMC to our data
    mcmc_run.run(data, latent_dim, random_mask)

    pyro.clear_param_store()
    #print(dummy_train)
    
    posterior_samples = mcmc_run.get_samples()

    # Extract W, sigma, and Z samples
    W_samples = posterior_samples["W"]
    sigma_samples = posterior_samples["sigma"]
    Z_samples = posterior_samples["Z"]
    
    return W_samples, sigma_samples, Z_samples

W_samples, sigma_samples, Z_samples = bayesian_pca_mcmc(dummy_train_x)

Sample: 100%|██████████| 300/300 [05:46,  1.16s/it, step size=1.41e-04, acc. prob=0.345]
